In [76]:
%%writefile test.py


import streamlit as st
import pandas as pd
from datetime import datetime 
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import plotly.graph_objects as go
import folium
from streamlit_folium import folium_static
import plotly.express as px

data = pd.read_excel("streamlit_data (1).xlsx")
# data['일자']= data['일자'].astype('str')
# data['일자'] = pd.to_datetime(data['일자'])

# 한글폰트 설정
font_path = "C:/Windows/Fonts/malgun.ttf"  # 원하는 한글 폰트 경로로 수정
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

# 레이아웃 구성하기 
st.set_page_config(layout="wide")

# 번역
def translate_text(text, target_language):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language).text
    return translated_text
 
# 첫 번째 탭 (KOR)
with st.sidebar:
    kor_tab = st.radio("Select Language (KOR/ENG)", ["KOR", "ENG"], key="kor_tab")

st.sidebar.header("Filters")
st.sidebar.date_input("Start date", data['일자'].min())
st.sidebar.date_input("End date", data['일자'].max())

#create a multiselect widget to display genre
new_genre_list = st.sidebar.multiselect('Choose topics:', data['topics'].unique(), default = ['전쟁'])
### topics = nan 값 제거하기!

### 지역 선택 제외

# 실제 내용
if kor_tab == "KOR":
    st.write("한국어 내용을 여기에 표시")
else:
    st.write("English content will be displayed here")

# tabs 만들기 
tab1, tab2 = st.tabs(["국내", "해외"])

# tab1 내용물 구성하기 
with tab1:

    # 제목 넣기
    st.markdown("## 뉴스빅데이터를 기반으로 재난/재해 상황을 한 눈에")

    ##### folium을 활용하여 지도 그리기
    m = folium.Map(location=[39.949610, -75.150282], zoom_start=16)
    st_data = folium_static(m, width=1000)

    # 한국 도시와 외국 도시 데이터를 나누어 저장
    korean_data = data[data['외국유무'] == '한국']
    foreign_data = data[data['외국유무'] == '외국']

    # 가로로 나란히 두 개의 열 생성
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.header("Topics 분포")
        topics_distribution = data['topics'].value_counts()
        fig_topics = px.pie(
            topics_distribution,
            names=topics_distribution.index,
            title='Topics 분포',
            hole=0.4  # 도넛 차트 설정
            )
        st.plotly_chart(fig_topics)
        
    # 가운데 열에는 한국 도시의 Plotly 차트 표시
    with col2:
        st.header("Top 5 한국 도시별 리스크")
        korean_top_5 = korean_data.groupby('city')['risk'].sum().nlargest(5)
        korean_top_5 = korean_top_5[::-1]  # 역순으로 정렬
        fig_korean = go.Figure(go.Bar(y=korean_top_5.index, x=korean_top_5.values, orientation='h'))
        st.plotly_chart(fig_korean)

    # 오른쪽 열에는 외국 도시의 Plotly 차트 표시
    with col3:
        st.header("Top 5 외국 도시별 리스크")
        foreign_top_5 = foreign_data.groupby('city')['risk'].sum().nlargest(5)
        foreign_top_5 = foreign_top_5[::-1]  # 역순으로 정렬
        fig_foreign = go.Figure(go.Bar(y=foreign_top_5.index, x=foreign_top_5.values, orientation='h'))
        st.plotly_chart(fig_foreign)

    col1, col2 = st.columns([70, 30])
    with col1:
        st.dataframe(data[['일자', '제목', '요약']])


Overwriting test.py
